In [1]:
# Entity Extraction and Coerefernce resolution
variable_name = "yhat_references"
max_new_tokens=250
temperature=0.7

https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ/discussions/11

In [ ]:
#!pip install sentencepiece
#!pip install tokenizers -U
#!pip install transformers[sentencepiece] -U
#!pip install accelerate -U
#!pip install optimum

#https://huggingface.co/TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ/discussions/5



In [2]:
import os

#https://stackoverflow.com/questions/73747731/runtimeerror-cuda-out-of-memory-how-setting-max-split-size-mb
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64" #experiment with this and the value, see if it prevents oom

# Get the current working directory
cwd = os.getcwd()
cwd
os.chdir('/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/')
cwd = os.getcwd()
cwd

'/media/skynet3/8tb_a/rwd_github_private/ICBeLLM'

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #auto gets confused for some reason and puts it on the 2080
import torch
torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()
torch.cuda.get_device_name(device=None)

import torch
torch.cuda.empty_cache()

In [4]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# If you've already downloaded the model, reference its location here:
#quantized_model_dir = "/path/to/TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"
# Or to download it from the hub and store it in the Hugging Face cache directory:

#quantized_model_dir = "./data_temp/Wizard-Vicuna-13B-Uncensored-GPTQ/" #TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ
#quantized_model_dir = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ"
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=False)


/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#FileNotFoundError: Could not find model in TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ
#https://huggingface.co/TheBloke/wizard-vicuna-13B-GPTQ/discussions/7
#quantized_model_dir = "TheBloke/stable-vicuna-13B-GPTQ"
#model_basename = "wizard-vicuna-13B-GPTQ-4bit.compat.no-act-order" #YOU HAVE TO DO THIS

#model_basename = "Wizard-vicuna-30B-Uncensored-GPTQ-4bit.act-order"
# path to directory containing local model
#https://huggingface.co/TheBloke/starcoderplus-GPTQ/discussions/3
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ" #"/workspace/models/TheBloke_Wizard-Vicuna-30B-Uncensored-GPTQ"
model_basename = "Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order" #.safetensors
use_triton = False 

model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, 
                                           device="cuda:0", 
                                           use_triton=False,
                                           use_safetensors=True , 
                                           model_basename=model_basename,
                                           quantize_config=None,
                                           #torch_dtype=torch.bfloat16, 
                                           trust_remote_code=True
                                           ) #





The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at /home/skynet3/.cache/huggingface/hub/models--TheBloke--Wizard-Vicuna-30B-Uncensored-GPTQ/snapshots/7aeb8db6d49732afa76dc7ce60c5c040dbbcf3ea/Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [6]:
model.hf_device_map #yup, it offloaded the last layers to the CPU. That's why it's so slow.

{'': 0}

In [7]:
prompt = "Ask Something here"
prompt_model = f'''### Human: {prompt}
### Assistant:'''
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

print("*** Output ***")
print(tokenizer.decode(output[0]))
print("**************")

*** Output ***
<s> ### Human: Ask Something here
### Assistant: Sure, I can help you with anything related to the topic of this book. What would you like to know?</s>
**************


In [ ]:
# Ok it continues to be about 4 characters per token
# print(input_ids.shape) #So this prompt and a little bit of story is about 939 tokens, got another 1k to play with
#print(len(prompt))
#5046/1287


# Tune the prompt

In [8]:
prompt_base = """You are a natural language processing pipeline. You read a paragraph of text and then perform a task on final sentence. The task is to create a semicolon separated list of entities mentioned in final sentence. Perform coreference resolution and map ambiguous references to named entities mentioned earlier in the paragraph. Think step by step. Explain your reasoning and then give a final answer.

### Paragraph:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Final Sentence:
The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Thought:
The final sentence to process is "The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok."
The references in that final sentence are "Western Allies", "Germany", "eastern front", and "Vladivostok".
"Western Allies" refers to a coalition of countries who are not explicitly listed yet in the paragraph and so include "Western Allies" in the list.
"Germany" directly refers to a country and so include "Germany" in the list.
"eastern front" indirectly refers to the country "Russia" so include "Russia" in the list.
"Vladivostok" directly refers to a city so include "Vladivostok" in the list.

### Final Answer:
Western Allies; Germany; Russia; Vladivostok

### Paragraph:
Three of the Austro-Hungarian Empire's successor states--Czechoslovakia, Yugoslavia, and Hungary--experienced a two-phase crisis from 27 March until 15 November 1921. Karl IV, the last Hapsburg Emperor of Austria-Hungary, abdicated in November 1918 and was exiled to Switzerland; but he did not abdicate his right to the throne. Early in 1921 the Conference of Ambassadors, acting for the Supreme Council (Big Ten) at the Paris Peace Conference, issued a decree forbidding the restoration of the Hapsburg family to the vacant Hungarian throne. Moreover, several European governments issued statements indicating what recourse they might have if a Hapsburg reascended the throne in Budapest. On 27 March 1921 Karl unexpectedly returned to his homeland and called upon the Regent of Hungary, Admiral Horthy, to transfer his powers to the rightful ruler. This act triggered a crisis for Czechoslovakia and Yugoslavia.

### Final Sentence:
The latter responded two days later by declaring that it would regard any attempt to restore Karl as a casus belli.

### Thought:
The final sentence to process is "The latter responded two days later by declaring that it would regard any attempt to restore Karl as a casus belli."
The references in that final sentence are "The latter", "attempt to restore Karl", and "Karl".
"The latter' indirectly refers to the entity "Yugoslavia" from the previous sentence so include "Yugoslavia" in the list.
"any attempt to restore Karl" indirectly refers to an action by the country "Hungary" so include Hungary in the list.
"Karl" refers to the entity "Karl IV" introduced earlier in the paragraph so include "Karl IV" in the list.

### Final Answer:
Yugoslavia; Hungary; Karl IV

### Paragraph:
Cuba and the United States were the adversaries in a crisis from 15 to 24 April 1961. A series of incursions into Caribbean and Central American states, in which Cuba was perceived to be involved, took place in 1959-60 In addition, the U.S. had become increasingly concerned about the strong ties that had developed between Cuba and the USSR. Relations between the U.S. and Cuba were strained; and the latter appealed to the Security Council at the end of 1960 charging the U.S. with plans to mount an invasion of Cuba using Cuban exiles trained in Guatemala. Diplomatic relations between the two countries were broken on 3 January 1961. Cuba and the United States were the adversaries in a crisis from 15 to 24 April 1961. A series of incursions into Caribbean and Central American states, in which Cuba was perceived to be involved, took place in 1959-60 In addition, the U.S. had become increasingly concerned about the strong ties that had developed between Cuba and the USSR. Relations between the U.S. and Cuba were strained; and the latter appealed to the Security Council at the end of 1960 charging the U.S. with plans to mount an invasion of Cuba using Cuban exiles trained in Guatemala. Diplomatic relations between the two countries were broken on 3 January 1961.

### Final Sentence:
In early April 1961 President Kennedy authorized an invasion of Cuba by the exiles, reasoning that, in the event of a failure, the remaining force would be able to establish guerrilla bases in the Cuban mountains.

### Thought: 
The final sentence to process is "In early April 1961 President Kennedy authorized an invasion of Cuba by the exiles, reasoning that, in the event of a failure, the remaining force would be able to establish guerrilla bases in the Cuban mountains."
The references in that final sentence are "President Kennedy", "Cuba", "exiles", "guerrilla bases", and "Cuban mountains".
"President Kennedy" refers to the entity "President John F. Kennedy" and so include "President John F. Kennedy" in the list.
"Cuba" directly refers to the country "Cuba" and so include "Cuba" in the list.
"exiles" refers to the group "Cuban exiles" introduced earlier in the paragraph and so include "Cuban exiles" in the list.
"Cuban mountains" directly refers to a geographic feature "Cuban mountains" so include "Cuban exiles" in the list.

### Final Answer:
President John F. Kennedy; Cuba; Cuban exiles; Cuban mountains

### Paragraph:"""

this_obs = """
India experienced a crisis over Hyderabad from 21 August to 18 September 1948. Hyderabad was the second largest princely state in the subcontinent, after Jammu and Kashmir. It is located in the geographic center of India and, in 1948, 80 percent of its population was Hindu while most of the political elite were Muslim. The nizam of Hyderabad did not favor accession to distant Pakistan.

### Final Sentence:
Nor did predominantly Hindu India attract him.
"""
prompt = prompt_base + this_obs
prompt_model = f'''### Human: {prompt}
### Assistant:'''
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
#print(input_ids.shape) #Ok if we pass too long a prompt it goes OOM. We have to catch that. #2048
assert(input_ids.shape[1]+250 < 2048 )

In [12]:
input_ids.shape[1]+250 < 2048

True

In [35]:
print(prompt_model)

### Human: You are a natural language processing pipeline. You read a paragraph of text and then perform a task on final sentence. The task is to create a semicolon separated list of entities mentioned in final sentence. Perform coreference resolution and map ambiguous references to named entities mentioned earlier in the paragraph. Think step by step. Explain your reasoning and then give a final answer.

### Paragraph:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Final Sentence:
The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Thought:
The final sentence to process is "The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vl

In [13]:
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
#print(input_ids.shape) #Ok if we pass too long a prompt it goes OOM. We have to catch that. #2048
assert(input_ids.shape[1]+250 < 2048 )
output = model.generate(inputs=input_ids, temperature=temperature, max_new_tokens=max_new_tokens) #uh oh, 1966 was too big too. Do both the max tokens and prompt have to fit in the context window?
answer = tokenizer.decode(output[0])

In [ ]:
print(answer)

In [15]:
print(answer.replace(prompt,'').replace('<s>### Human: \n','').replace('<s> ### Human: ','').replace('\n### Assistant:',''))


The final sentence to process is "Nor did predominantly Hindu India attract him."
The references in that final sentence are "India" and "Hindu India".
"India" directly refers to the country "India" and so include "India" in the list.
"Hindu India" indirectly refers to the religion "Hinduism" and so include "Hinduism" in the list.

### Final Answer:
India; Hinduism</s>


In [ ]:
import pandas as pd
import os
max_new_tokens=250
temperature=0.7

ICBeLLM_xy = pd.read_csv("/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/ICBeLLM_xy.tsv", sep="\t", na_values='').reset_index(drop=True).fillna('')
ICBeLLM_xy.story_so_far = ICBeLLM_xy.story_so_far.fillna('')
file="/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/" + variable_name + ".tsv"
if os.path.exists(file):
    temp=pd.read_csv(file,sep="\t", na_values='').fillna('')
else:
    temp=ICBeLLM_xy.copy()
    temp.loc[:,'model']=quantized_model_dir
    temp.loc[:,'max_new_tokens']=max_new_tokens
    temp.loc[:,'temperature']=temperature
    temp.loc[:,'variable']=variable_name
    temp.loc[:,'time'] = None
    temp.loc[:,'n_tokens'] = None
    temp.loc[:,'prompt']=None
    temp.loc[:,'response']=None

In [ ]:
max_new_tokens=250
i=1
prompt = prompt_base + temp['story_so_far'].values[i] + "\n\n### Final Sentence:\n" + "\n"
prompt_model = f'''### Human: {prompt}
### Assistant:'''
tokens_remaining = 2048 - max_new_tokens - len(prompt_model)/4
print(tokens_remaining)

In [ ]:
import numpy as np
characters_remaining = np.round((tokens_remaining*4))
#print(characters_remaining)
n=len(temp['sentence_span_text'].values[i])
#print(n)
start=max(n-characters_remaining, 0)
temp['sentence_span_text'].values[i][start:n]


In [ ]:
195-60

In [ ]:
#prompt = prompt_base + temp['story_so_far'].values[i] + "\n\n### Final Sentence:\n" + temp['sentence_span_text'].values[i][-(tokens_remaining*4):] + "\n"
#prompt_model = f'''### Human: {prompt}
#### Assistant:'''
#
#print(prompt_model)
#input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
#print(input_ids.shape) #Ok if we pass too long a prompt it goes OOM. We have to catch that. #2048
#assert(input_ids.shape[0]+250 < 2048 )


In [ ]:
i=618
int(i)
prompt = prompt_base + temp['sentence_span_text'].values[i] + "\n\n### Final Sentence:\n" + "\n"
prompt_model = f'''### Human: {prompt}
### Assistant:'''
tokens_remaining = 2048 - max_new_tokens - len(prompt_model)/4 - 150 #750 #150 #take off another buffer 150 characters just to be safe
characters_remaining = np.round((tokens_remaining*4))
n=len(temp['story_so_far'].values[i])
print(n)
start= int(max(n-characters_remaining, 0))
print(start)
prompt = prompt_base + "\n" + temp['story_so_far'].values[i][start:n] + "\n\n### Final Sentence:\n" + temp['sentence_span_text'].values[i] + "\n"
prompt_model = f'''### Human: {prompt}
### Assistant:'''
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
print(input_ids.shape[1]+max_new_tokens)
assert(input_ids.shape[1]+max_new_tokens < 2048 ) #I'm getting OOM errors at least at 1961, 1629 is ok
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=max_new_tokens) #uh oh, 1966 was too big too. Do both the max tokens and prompt have to fit in the context window?
answer = tokenizer.decode(output[0])
temp.loc[i,'prompt'] = prompt_model
temp.loc[i,'response'] = answer.replace(prompt_model,'')

#temp.to_csv(file, sep="\t", index=False)

2358
861
1982


In [ ]:
input_ids.shape[1] + max_new_tokens

1982

In [ ]:
import timeit
import numpy as np
#65 fails
#Grab the last 2k characters of the story to avoid oom
for i in range(temp.shape[0]):
    if pd.isna(temp['response'][i]) or temp['response'][i]=='' :
        start_time = timeit.default_timer()
        #print(i)
        prompt = prompt_base + temp['sentence_span_text'].values[i] + "\n\n### Final Sentence:\n" + "\n"
        prompt_model = f'''### Human: {prompt}
        ### Assistant:'''
        tokens_remaining = 2048 - max_new_tokens - len(prompt_model)/4 - 150 #take off 150 just to be safe
        characters_remaining = np.round((tokens_remaining*4))
        n=len(temp['story_so_far'].values[i])
        start= int(max(n-characters_remaining, 0))
        prompt = prompt_base + "\n" + temp['story_so_far'].values[i][start:n] + "\n\n### Final Sentence:\n" + temp['sentence_span_text'].values[i] + "\n"
        prompt_model = f'''### Human: {prompt}
        ### Assistant:'''
        input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
        assert(input_ids.shape[1]+max_new_tokens < 2048 )
        output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=max_new_tokens) #uh oh, 1966 was too big too. Do both the max tokens and prompt have to fit in the context window?
        answer = tokenizer.decode(output[0])
        stopt_time = timeit.default_timer() 
        temp.loc[i,'time'] = round(stopt_time- start_time,3)
        temp.loc[i,'n_tokens']=input_ids.shape[1]
        temp.loc[i,'prompt'] = prompt_model
        temp.loc[i,'response'] = answer.replace(prompt_model,'')
        
        temp.to_csv(file, sep="\t", index=False)
        #torch.cuda.empty_cache()

#ICBeLLM_xy['prompt_entity_list_explicit']
#[prompt]
#ICBeLLM_xy['story_so_far'].fillna('').values[i] 
#pd.isna(temp['yhat_prompt_entity_yes_no'][1])
#temp['yhat_prompt_entity_yes_no'][0] = "asdasdasd"


In [ ]:
i #618
#temp['story_so_far'].values[i]

618